<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/toNamedTuple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/dotmap.ipynb
!pip install importnb
from importnb import Notebook
with Notebook(): 
    from dotmap import DotMap
from collections import namedtuple
import jax
import numpy as np
import jax.numpy as jnp
import jax.tree_util as tu
from functools import partial

In [18]:
d=DotMap()
d.a.j=10.
d.a.k=20.

d.b.y=5.
d.b.x.m=2.
d.b.x.n=4.

d.c=4.

d=d.toDict()

In [19]:
def recurse(d,e={},root='r'):
    for k,v in d.items():
        if isinstance(v,Mapping):
            e[k]={}
            e[k]=recurse(v,e[k],root+"_"+k)
        else:
            e[k]=v
    NT=namedtuple(root,e.keys())
    return NT(**e)


In [20]:
recurse(d)

r(a=r_a(j=10.0, k=20.0), b=r_b(y=5.0, x=r_b_x(m=2.0, n=4.0)), c=4.0)

In [21]:
def toNamedTuple(d):
    stack=[(d,partial(d.__setitem__,'toNamedTupleResult'))]


    while stack:
        build={}
        nodict=True
        source,setter=stack[-1]
        for k,v in source.items():
            if isinstance(v,dict):
                stack.append((v, partial(source.__setitem__,k)))
                nodict=False
        if nodict:            
            NT = namedtuple('_', source.keys())
            setter(NT(**source))
            stack.pop()
            
    return d['toNamedTupleResult']

In [22]:
dt=toNamedTuple(d)
dt

_(a=_(j=10.0, k=20.0), b=_(y=5.0, x=_(m=2.0, n=4.0)), c=4.0)

In [27]:
val,tree = tu.tree_flatten(dt)

In [28]:
val,tree

([10.0, 20.0, 5.0, 2.0, 4.0, 4.0],
 PyTreeDef(CustomNode(namedtuple[<class '__main__._'>], [CustomNode(namedtuple[<class '__main__._'>], [*, *]), CustomNode(namedtuple[<class '__main__._'>], [*, CustomNode(namedtuple[<class '__main__._'>], [*, *])]), *])))

In [30]:
val2 = np.random.normal(size=len(val))

In [32]:
%%timeit
tu.tree_unflatten(tree, val2)

The slowest run took 23.96 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 4.55 µs per loop
